In [23]:
# Load the data
import csv
import pandas as pd

data_file = 'data/homo_test_file2.csv'
vehicle_file = 'data/homo_vehicle_file2.csv'

# Read the data file
data = pd.read_csv(data_file, sep=' ')

data


,ID,start_x,start_y,student_num,service_duration,school_d,school_y,school_tw_st,school_tw_end
0,1,2,84,31,33,65,16,91,110
1,2,78,29,59,37,65,16,91,110
2,3,90,67,53,31,65,16,91,110
3,4,95,97,55,24,4,45,150,178
4,5,68,75,39,28,4,45,150,178
5,6,27,62,47,27,4,45,150,178
6,7,16,33,50,35,4,45,150,178
7,8,97,13,45,34,4,45,150,178
8,9,61,60,63,23,4,45,150,178
9,10,85,67,52,31,4,45,150,178


In [24]:
vehicle_df = pd.read_csv(vehicle_file, sep=' ')
vehicle_df

,Veh_ID,depot_x,depot_y,capacity
0,1,50,50,70
1,2,50,50,70
2,3,50,50,70
3,4,50,50,70
4,5,50,50,70
5,6,50,50,70
6,7,50,50,70
7,8,50,50,70
8,9,50,50,70
9,10,50,50,70


In [25]:
# Load Gurobi
import gurobipy as gp
from gurobipy import GRB


In [67]:
import math

# Depot location is 50, 50,
# Capacity is 70
# Copied from https://github.com/ruthmair/vrp/blob/main/cvrp.ipynb

vehicle_capacity = 70

depot = 0
customers = [*data['ID'].values]
locations = [depot] + customers
connections = [(i, j) for i in locations for j in locations if i != j]
vehicles = [*vehicle_df['Veh_ID'].values]

# Depot at 50, 50
points = [(50, 50), *[(i[1]['start_x'], i[1]['start_y']) for i in data.iterrows()]]

costs = {
    (i, j): math.ceil(
        math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
    )
    for i in locations
    for j in locations
    if i != j
}

demands = {i[1]['ID']: i[1]['student_num'] for i in data.iterrows()}
demands[0] = 0 # Depot demand is 0

demands

{1: 31,
 2: 59,
 3: 53,
 4: 55,
 5: 39,
 6: 47,
 7: 50,
 8: 45,
 9: 63,
 10: 52,
 11: 34,
 12: 40,
 13: 43,
 0: 0}

In [2]:


def num_vehicles_needed_for_customers(custs):
    sum_demand = 0
    for i in custs:
        sum_demand += demands[i]
    return math.ceil(sum_demand / vehicle_capacity)



# Create a new model
m = gp.Model("CVRP")

x = m.addVars(connections, vtype=GRB.BINARY, name="x")

m.setObjective(x.prod(costs), GRB.MINIMIZE)

# Bus stop flow
m.addConstrs((x.sum("*", j) == 1 for j in customers), name="incoming")
m.addConstrs((x.sum(i, "*") == 1 for i in customers), name="outgoing")

# Vehicle number




Restricted license - for non-production use only - expires 2024-10-28


<gurobi.Constr *Awaiting Model Update*>

In [5]:

# Solve it!
m.optimize()


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: AMD Ryzen 7 3700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]


Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [4]:

print(f"Optimal objective value: {m.objVal}")
print(f"Solution values: x={x.X}, y={y.X}, z={z.X}")

Optimal objective value: 3.0
Solution values: x=1.0, y=0.0, z=1.0
